<a href="https://colab.research.google.com/github/phamthien300820/CS114.K21/blob/master/ML_RandomForest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
from tqdm import tqdm
import os
import cv2
import pandas as pd
import numpy as np
import h5py
!pip install mahotas
import mahotas
from sklearn.preprocessing import LabelEncoder 
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier


     |████████████████████████████████| 5.7MB 2.9MB/s 


In [3]:
#Đường dẫn output
output_path = "/content/drive/My Drive/X_i1"

# Đường dẫn dữ liệu training
train_path = "/content/drive/My Drive/Machine_Learning Helmet"

# lấy nhẵn training
train_labels = os.listdir(train_path)
train_labels.sort()

# Kích thước khi ảnh resize
fixed_size = tuple((100, 100))

bins = 8

# danh sách trống để giữ các feature vectors và nhãn 
global_features = []
labels = []

In [5]:
def fd_hu_moments(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    feature = cv2.HuMoments(cv2.moments(image)).flatten()
    return feature

In [6]:
def fd_haralick(image):
    #chuyển về ảnh gray
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    haralick = mahotas.features.haralick(gray).mean(axis=0)
    return haralick

In [7]:
def fd_histogram(image, mask=None):
    #chuyển đổi hình ảnh sang màu HSV
    image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hist  = cv2.calcHist([image], [0, 1, 2], None, [bins, bins, bins], [0, 256, 0, 256, 0, 256])
    cv2.normalize(hist, hist)
    return hist.flatten()

In [8]:
train_labels

['Có người đi xe máy', 'Không có xe máy']

In [9]:
for training_name in train_labels:
    
    dir = os.path.join(train_path, training_name)
    # nhãn đang được train hiện tại
    current_label = training_name
    # Duyệt tất cả các ảnh trong mỗi tập con
    for image in tqdm(os.listdir(dir)):
        
        try:
          path = os.path.join(dir,image)
          #Đọc ảnh và resize lại theo fixed_size
          image = cv2.imread(path)
          image = cv2.resize(image, fixed_size)

          
          fv_hu_moments = fd_hu_moments(image)
          fv_haralick   = fd_haralick(image)
          fv_histogram  = fd_histogram(image)

          
          global_feature = np.hstack([fv_histogram, fv_hu_moments, fv_haralick])

          # cập nhật danh sách các nhãn và feature vectors
          labels.append(current_label)
          global_features.append(global_feature)
        except:
            pass

    print("[STATUS] processed folder: {}".format(current_label))

print("[STATUS] completed Global Feature Extraction...")

le = LabelEncoder()
target = le.fit_transform(labels)

scaler = MinMaxScaler(feature_range=(0, 1))
rescaled_features = scaler.fit_transform(global_features)

# lưu feature vector bằng HDF5
h5f_data = h5py.File(output_path+'data.h5', 'w')
h5f_data.create_dataset('dataset_1', data=np.array(rescaled_features))

h5f_label = h5py.File(output_path+'labels.h5', 'w')
h5f_label.create_dataset('dataset_1', data=np.array(target))

h5f_data.close()
h5f_label.close()

print("[STATUS] end of training..")

  0%|          | 0/1221 [00:00<?, ?it/s]

[STATUS] processed folder: Có người đi xe máy


100%|██████████| 1221/1221 [13:31<00:00,  1.50it/s]

[STATUS] processed folder: Không có xe máy
[STATUS] completed Global Feature Extraction...
[STATUS] end of training..


In [11]:
output_path = "\content\X"

# Số lượng cây ở RandomForest
num_trees = 300

bins = 8

# số hình ảnh mỗi lớp
images_per_class = 10;

# nhập vector đặc trưng và nhãn được training
h5f_data = h5py.File('/content/drive/My Drive/X_i1data.h5', 'r')
h5f_label = h5py.File('/content/drive/My Drive/X_i1labels.h5', 'r')

global_features_string = h5f_data['dataset_1']
global_labels_string = h5f_label['dataset_1']

global_features = np.array(global_features_string)
global_labels = np.array(global_labels_string)

h5f_data.close()
h5f_label.close()


#  Tạo model - Random Forests
clf  = RandomForestClassifier(n_estimators=num_trees)
clf.fit(global_features, global_labels)

# Đường dẫn đến dữ liệu test
test_path = "/content/drive/My Drive/Train_1"
# lấy labels của test
test_labels = os.listdir(test_path)


test_labels.sort()
print(test_labels)

test_features = []
test_results = []
for testing_name in test_labels:
    dir = os.path.join(test_path, testing_name)
    current_label = testing_name
    for image in tqdm(os.listdir(dir)):
        try:
          path = os.path.join(dir,image)
          # đọc hình ảnh và resize của nó với fixed-size
          image = cv2.imread(path)
          image = cv2.resize(image, fixed_size)


          fv_hu_moments = fd_hu_moments(image)
          fv_haralick   = fd_haralick(image)
          fv_histogram  = fd_histogram(image)

          test_results.append(current_label)
          test_features.append(np.hstack([fv_histogram, fv_hu_moments, fv_haralick]))
        except:
            pass

# Dự đoán nhãn của ảnh thử nghiệm
le = LabelEncoder()
y_result = le.fit_transform(test_results)
y_pred = clf.predict(test_features)

print(y_pred)
print("Kết quả: ", (y_pred == y_result).tolist().count(True)/len(y_result))

0it [00:00, ?it/s]
  0%|          | 0/182 [00:00<?, ?it/s]

['.ipynb_checkpoints', 'Có người đi xe máy', 'Không có xe máy']


100%|██████████| 163/163 [00:26<00:00,  6.12it/s]


[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0
 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1 1 1 0
 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 0 1 0
 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 0 0 0 0 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 0 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 0 1 0 0 1 1
 1 1 1 1 1 1 1 0 1 0 0 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 1 0 1 0 0 0 0 0
 1 1 1 1 1 0 1 0]
Kết quả:  0.4681647940074906


In [12]:
from sklearn.metrics import classification_report
print(classification_report(y_result, y_pred, labels=[0, 1]))

              precision    recall  f1-score   support

           0       0.48      0.16      0.24       140
           1       0.47      0.80      0.59       127

    accuracy                           0.47       267
   macro avg       0.47      0.48      0.42       267
weighted avg       0.47      0.47      0.41       267



Lưu model và test lại:

In [20]:
import pickle
classifier_file = "model1.h5"
with open(classifier_file, 'wb') as outfile:
                pickle.dump((clf), outfile)

In [26]:
import cv2,os
loaded_model = pickle.load(open('/content/drive/My Drive/RDC.h5', 'rb'))

In [27]:
h5f_data = h5py.File('/content/drive/My Drive/X_i1data.h5', 'r')
h5f_label = h5py.File('/content/drive/My Drive/X_i1labels.h5', 'r')

global_features_string = h5f_data['dataset_1']
global_labels_string = h5f_label['dataset_1']

global_features = np.array(global_features_string)
global_labels = np.array(global_labels_string)

h5f_data.close()
h5f_label.close()

# Đường dẫn đến dữ liệu test
test_path = "/content/drive/My Drive/Train_1"
# lấy labels của test
test_labels = os.listdir(test_path)


test_labels.sort()
print(test_labels)

test_features = []
test_results = []
for testing_name in test_labels:
    dir = os.path.join(test_path, testing_name)
    current_label = testing_name
    for image in tqdm(os.listdir(dir)):
        try:
          path = os.path.join(dir,image)
          # đọc hình ảnh và resize của nó với fixed-size
          image = cv2.imread(path)
          image = cv2.resize(image, fixed_size)


          fv_hu_moments = fd_hu_moments(image)
          fv_haralick   = fd_haralick(image)
          fv_histogram  = fd_histogram(image)

          test_results.append(current_label)
          test_features.append(np.hstack([fv_histogram, fv_hu_moments, fv_haralick]))
        except:
            pass

# Dự đoán nhãn của ảnh thử nghiệm
le = LabelEncoder()
y_result = le.fit_transform(test_results)
y_pred = loaded_model.predict(test_features)

print(y_pred)
print("Kết quả: ", (y_pred == y_result).tolist().count(True)/len(y_result))

0it [00:00, ?it/s]
  0%|          | 0/182 [00:00<?, ?it/s]

['.ipynb_checkpoints', 'Có người đi xe máy', 'Không có xe máy']


100%|██████████| 163/163 [00:26<00:00,  6.20it/s]


[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0
 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1 1 1 0
 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 0 1 0
 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 0 0 0 0 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 0 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 0 1 0 0 1 1
 1 1 1 1 1 1 1 0 1 0 0 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 1 0 1 0 0 0 0 0
 1 1 1 1 1 0 1 0]
Kết quả:  0.4681647940074906
